In [1]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

In [2]:
def load_pts_file(pts_path):
    with open(pts_path, 'r') as f:
        lines = f.readlines()
    start = lines.index('{\n') + 1
    end = lines.index('}\n')
    landmarks = []
    for line in lines[start:end]:
        x, y = map(float, line.strip().split())
        landmarks.append([x, y])
    return np.array(landmarks, dtype=np.float32)

In [3]:
class AFLWStyleDataset(Dataset):
    def __init__(self, list_file, transform=None):
        self.samples = []
        self.transform = transform

        with open(list_file, 'r') as f:
            for line in f:
                if line.strip() == '':
                    continue
                parts = line.strip().split()
                image_path = parts[0]
                pts_path = parts[1]
                bbox = list(map(float, parts[2:6]))
                face_size = float(parts[6])
                self.samples.append({
                    'image_path': image_path,
                    'pts_path': pts_path,
                    'bbox': bbox,
                    'face_size': face_size
                })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        image = Image.open(sample['image_path']).convert('RGB')
        landmarks = load_pts_file(sample['pts_path'])  # shape (19, 2)
        landmarks = torch.from_numpy(landmarks)        # shape (19, 2)
        bbox = torch.tensor(sample['bbox'], dtype=torch.float32)
        face_size = torch.tensor(sample['face_size'], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return {
            'image': image,
            'landmarks': landmarks,
            'bbox': bbox,
            'face_size': face_size,
            'path': sample['image_path']
        }


In [4]:
from torch.utils.data import DataLoader
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),  # Converts to [0,1] tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Optional
])

list_path = 'lists/AFLW/Original/train.GTL'  # or test.GTB, etc.
dataset = AFLWStyleDataset(list_file=list_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)


FileNotFoundError: [Errno 2] No such file or directory: 'lists/AFLW/Original/train.GTL'

In [5]:
for batch in dataloader:
    images = batch['image']           # shape (B, 3, 256, 256)
    landmarks = batch['landmarks']    # shape (B, 19, 2)
    bboxes = batch['bbox']            # shape (B, 4)
    face_sizes = batch['face_size']   # shape (B,)
    paths = batch['path']
    break  # just inspect first batch


NameError: name 'dataloader' is not defined